In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
%matplotlib inline

## The Data

In [2]:
train_normal = pd.read_csv('./data/normal_training.csv')

In [3]:
test_faulty = pd.read_csv('./data/faulty_testing.csv')

In [4]:
# data preparing
df_train = train_normal[(train_normal.simulationRun>=1)].iloc[:500,3:]

df_test = train_normal[(train_normal.simulationRun>5)&(train_normal.simulationRun<10)].iloc[:,3:].reset_index(drop=True)
# IDV = 1
# df_test = test_faulty[(test_faulty.faultNumber==IDV) & 
#                                (test_faulty.simulationRun==1)].iloc[:,3:]

In [5]:
def scl_pick(name, data):
    """
    the several option for name:
    -std
    -minmax
    -robust
    -norm
    -pwr
    """
    if name == 'std':
        std_scl = StandardScaler().fit(data)
        df_scl = std_scl.transform(data)
        df_scl = pd.DataFrame(df_scl)
    elif name == 'minmax':
        MM_scl = MinMaxScaler().fit(data)
        df_scl = MM_scl.transform(data)
        df_scl = pd.DataFrame(df_scl)
    elif name == 'robust':
        rb_scl = RobustScaler().fit(data)
        df_scl = rb_scl.transform(data)
        df_scl = pd.DataFrame(df_scl)
    elif name == 'norm': ##### xxxxxx
        df_norm = Normalizer().fit(data)
        df_scl = df_norm.transform(data)
        df_scl = pd.DataFrame(df_scl)
    else:
        pt_scl = PowerTransformer().fit(data)
        df_scl = pt_scl.transform(data)
        df_scl = pd.DataFrame(df_scl)
    return df_scl

In [6]:
#-std-minmax-robust-norm-pwr
df_scl = scl_pick('std', df_train) #robust
df_scl = round(df_scl, 4)

## Training Centraling Kernel

In [7]:
# training proccess
A = df_scl
K = np.matmul(A, A.T)
K = round(K, 4)
K = pd.DataFrame(K)

C:\Users\m477a\AppData\Local\Temp/ipykernel_14368/409773423.py:3: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  K = np.matmul(A, A.T)


In [8]:
K.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,11.2548,-1.0064,0.0676,-2.9111,7.1808,2.8011,0.0056,1.4509,0.4064,1.5799,...,0.5931,3.8501,-9.1363,4.2920,-3.3997,-10.6721,-3.1037,5.0832,-1.1396,-2.0656
1,-1.0064,15.6484,-1.6438,5.6559,2.5905,-0.4405,6.0025,4.6070,-2.4341,2.6041,...,0.9103,1.0259,-2.7118,0.1543,0.3193,8.1392,5.3727,-5.6366,-8.4340,-1.2429
2,0.0676,-1.6438,26.7534,17.8788,-0.3142,5.1253,3.0534,9.6957,-6.1013,3.9280,...,6.0355,5.8826,9.4580,-6.5921,5.7674,-1.7370,-1.2486,-3.1192,-1.1443,-3.6522
3,-2.9111,5.6559,17.8788,28.0760,-1.2178,1.0283,7.3694,7.8789,-6.3967,-0.0438,...,4.3933,5.6325,7.7206,0.8120,2.4678,7.4409,0.0876,-8.0283,-7.5141,1.3204
4,7.1808,2.5905,-0.3142,-1.2178,40.4630,34.7837,-1.6995,-4.6649,5.3079,4.0270,...,2.6494,6.9232,-7.3993,0.6237,1.2561,-2.8090,-0.0294,7.9712,-3.6729,2.6613


In [9]:
s = (len(df_scl),len(df_scl))
unit = np.ones(s)/len(df_scl)

In [10]:
#######
unit = pd.DataFrame(unit)

In [11]:
K_c = K-unit@K-K@unit+unit@K@unit

In [12]:
K_c

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,11.254804,-1.006399,0.067607,-2.911104,7.180790,2.801094,0.005601,1.450910,0.406392,1.579916,...,0.593088,3.850108,-9.136295,4.292013,-3.399712,-10.672104,-3.103700,5.083205,-1.139614,-2.065601
1,-1.006399,15.648399,-1.643795,5.655894,2.590487,-0.440508,6.002499,4.607007,-2.434111,2.604113,...,0.910285,1.025905,-2.711797,0.154310,0.319285,8.139194,5.372698,-5.636598,-8.434016,-1.242903
2,0.067607,-1.643795,26.753411,17.878799,-0.314207,5.125297,3.053404,9.695713,-6.101305,3.928019,...,6.035491,5.882611,9.458008,-6.592084,5.767391,-1.737001,-1.248596,-3.119192,-1.144310,-3.652198
3,-2.911104,5.655894,17.878799,28.075988,-1.217818,1.028286,7.369393,7.878902,-6.396716,-0.043792,...,4.393280,5.632500,7.720597,0.812005,2.467780,7.440888,0.087593,-8.028303,-7.514122,1.320391
4,7.180790,2.590487,-0.314207,-1.217818,40.462976,34.783680,-1.699513,-4.664904,5.307878,4.027002,...,2.649374,6.923194,-7.399309,0.623699,1.256074,-2.809018,-0.029414,7.971191,-3.672928,2.661285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-10.672104,8.139194,-1.737001,7.440888,-2.809018,-4.698614,1.355193,-0.148798,3.515184,8.046708,...,-0.764420,-4.210900,3.462097,-6.719095,18.034280,40.463588,7.860093,-8.158003,4.955378,8.036091
496,-3.103700,5.372698,-1.248596,0.087593,-0.029414,-1.633409,-1.137002,0.502606,-8.135812,-5.761088,...,2.460984,2.056504,3.404502,-1.032691,-0.650716,7.860093,29.333497,19.186501,0.829183,-3.456904
497,5.083205,-5.636598,-3.119192,-8.028303,7.971191,2.713695,-6.155798,-6.842089,-4.205207,-12.150483,...,-2.619311,4.996009,6.710706,11.294114,-6.044211,-8.158003,19.186501,36.603206,9.409287,3.965700
498,-1.139614,-8.434016,-1.144310,-7.514122,-3.672928,-1.294024,-4.383217,-4.969508,2.194474,-4.847502,...,-4.139230,-8.862910,6.183287,-2.552405,6.351270,4.955378,0.829183,9.409287,33.992669,24.624281


## Manual EigenValues and EigenVector

In [13]:
K_c

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,11.254804,-1.006399,0.067607,-2.911104,7.180790,2.801094,0.005601,1.450910,0.406392,1.579916,...,0.593088,3.850108,-9.136295,4.292013,-3.399712,-10.672104,-3.103700,5.083205,-1.139614,-2.065601
1,-1.006399,15.648399,-1.643795,5.655894,2.590487,-0.440508,6.002499,4.607007,-2.434111,2.604113,...,0.910285,1.025905,-2.711797,0.154310,0.319285,8.139194,5.372698,-5.636598,-8.434016,-1.242903
2,0.067607,-1.643795,26.753411,17.878799,-0.314207,5.125297,3.053404,9.695713,-6.101305,3.928019,...,6.035491,5.882611,9.458008,-6.592084,5.767391,-1.737001,-1.248596,-3.119192,-1.144310,-3.652198
3,-2.911104,5.655894,17.878799,28.075988,-1.217818,1.028286,7.369393,7.878902,-6.396716,-0.043792,...,4.393280,5.632500,7.720597,0.812005,2.467780,7.440888,0.087593,-8.028303,-7.514122,1.320391
4,7.180790,2.590487,-0.314207,-1.217818,40.462976,34.783680,-1.699513,-4.664904,5.307878,4.027002,...,2.649374,6.923194,-7.399309,0.623699,1.256074,-2.809018,-0.029414,7.971191,-3.672928,2.661285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-10.672104,8.139194,-1.737001,7.440888,-2.809018,-4.698614,1.355193,-0.148798,3.515184,8.046708,...,-0.764420,-4.210900,3.462097,-6.719095,18.034280,40.463588,7.860093,-8.158003,4.955378,8.036091
496,-3.103700,5.372698,-1.248596,0.087593,-0.029414,-1.633409,-1.137002,0.502606,-8.135812,-5.761088,...,2.460984,2.056504,3.404502,-1.032691,-0.650716,7.860093,29.333497,19.186501,0.829183,-3.456904
497,5.083205,-5.636598,-3.119192,-8.028303,7.971191,2.713695,-6.155798,-6.842089,-4.205207,-12.150483,...,-2.619311,4.996009,6.710706,11.294114,-6.044211,-8.158003,19.186501,36.603206,9.409287,3.965700
498,-1.139614,-8.434016,-1.144310,-7.514122,-3.672928,-1.294024,-4.383217,-4.969508,2.194474,-4.847502,...,-4.139230,-8.862910,6.183287,-2.552405,6.351270,4.955378,0.829183,9.409287,33.992669,24.624281


In [14]:
K_c/len(df_scl)

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.022510,-0.002013,0.000135,-0.005822,0.014362,0.005602,0.000011,0.002902,0.000813,0.003160,...,0.001186,0.007700,-0.018273,0.008584,-0.006799,-0.021344,-0.006207,0.010166,-0.002279,-0.004131
1,-0.002013,0.031297,-0.003288,0.011312,0.005181,-0.000881,0.012005,0.009214,-0.004868,0.005208,...,0.001821,0.002052,-0.005424,0.000309,0.000639,0.016278,0.010745,-0.011273,-0.016868,-0.002486
2,0.000135,-0.003288,0.053507,0.035758,-0.000628,0.010251,0.006107,0.019391,-0.012203,0.007856,...,0.012071,0.011765,0.018916,-0.013184,0.011535,-0.003474,-0.002497,-0.006238,-0.002289,-0.007304
3,-0.005822,0.011312,0.035758,0.056152,-0.002436,0.002057,0.014739,0.015758,-0.012793,-0.000088,...,0.008787,0.011265,0.015441,0.001624,0.004936,0.014882,0.000175,-0.016057,-0.015028,0.002641
4,0.014362,0.005181,-0.000628,-0.002436,0.080926,0.069567,-0.003399,-0.009330,0.010616,0.008054,...,0.005299,0.013846,-0.014799,0.001247,0.002512,-0.005618,-0.000059,0.015942,-0.007346,0.005323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.021344,0.016278,-0.003474,0.014882,-0.005618,-0.009397,0.002710,-0.000298,0.007030,0.016093,...,-0.001529,-0.008422,0.006924,-0.013438,0.036069,0.080927,0.015720,-0.016316,0.009911,0.016072
496,-0.006207,0.010745,-0.002497,0.000175,-0.000059,-0.003267,-0.002274,0.001005,-0.016272,-0.011522,...,0.004922,0.004113,0.006809,-0.002065,-0.001301,0.015720,0.058667,0.038373,0.001658,-0.006914
497,0.010166,-0.011273,-0.006238,-0.016057,0.015942,0.005427,-0.012312,-0.013684,-0.008410,-0.024301,...,-0.005239,0.009992,0.013421,0.022588,-0.012088,-0.016316,0.038373,0.073206,0.018819,0.007931
498,-0.002279,-0.016868,-0.002289,-0.015028,-0.007346,-0.002588,-0.008766,-0.009939,0.004389,-0.009695,...,-0.008278,-0.017726,0.012367,-0.005105,0.012703,0.009911,0.001658,0.018819,0.067985,0.049249


In [15]:
u, s, vh = np.linalg.svd(K_c/len(df_scl))

In [16]:
u = pd.DataFrame(u)

In [17]:
# Unitary arrays (matrix)
# where u and the Hermitian 
# transpose of vh are 2D arrays with orthonormal columns
u

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.006387,0.023537,-0.004673,0.027282,-0.048683,0.001987,-0.036125,0.017959,-0.008994,0.032167,...,-0.050937,0.041807,-0.043418,0.005216,-0.037156,0.011797,-0.027347,0.005959,-0.039187,-0.044721
1,-0.010837,0.005218,0.014919,-0.048530,-0.016666,0.002775,0.009661,-0.019979,0.034658,-0.048371,...,-0.052567,-0.003085,-0.015609,-0.082312,0.042567,0.070397,-0.067255,-0.004390,0.052487,-0.044721
2,-0.009057,0.006176,0.027448,0.044896,-0.030452,-0.072360,0.011022,-0.010565,0.032952,-0.039450,...,0.004636,-0.046356,-0.023842,-0.024329,0.040072,0.008014,0.006137,-0.099489,0.025765,-0.044721
3,-0.010602,0.003314,0.021919,0.007954,-0.000284,-0.025181,0.000550,0.002075,0.055837,-0.063263,...,0.001097,0.022137,-0.016339,0.050687,-0.083193,0.041899,-0.013406,-0.034253,0.014198,-0.044721
4,-0.017506,0.042395,0.023584,0.022463,-0.041785,0.051631,-0.039164,0.036588,-0.051554,-0.017074,...,0.004793,-0.101694,-0.037562,0.037930,-0.098179,0.006870,0.038308,0.023583,0.044797,-0.044721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.015570,-0.035488,0.021553,-0.036509,0.071049,0.038697,0.063405,0.004241,0.037852,-0.082169,...,-0.037931,0.004436,-0.019700,-0.021188,0.028748,0.064265,-0.015115,-0.079277,0.016957,-0.044721
496,-0.003109,-0.035706,0.000365,-0.074290,-0.008611,-0.058478,-0.036679,-0.032770,0.021545,-0.004079,...,-0.037572,-0.023772,0.056510,-0.007092,-0.069547,-0.085795,-0.018858,-0.029559,0.017190,-0.044721
497,0.011545,-0.017246,-0.037298,-0.015249,-0.027140,-0.048940,-0.110584,0.014648,-0.031744,0.004925,...,0.026626,-0.015974,-0.028354,0.054069,0.061694,-0.050347,-0.031000,-0.001908,-0.065878,-0.044721
498,-0.002565,-0.008398,-0.042501,0.032443,0.023240,-0.018927,-0.011876,-0.003614,-0.064240,0.003310,...,-0.067470,0.027025,0.043941,-0.022657,-0.044801,-0.040020,0.013491,-0.037706,-0.037748,-0.044721


In [18]:
u.shape

(500, 500)

In [19]:
# singular values (vector)
# is a 1D array of a’s singular values
s

array([7.41572790e+00, 4.87716031e+00, 2.91702285e+00, 2.26304325e+00,
       2.09174059e+00, 1.96410904e+00, 1.91524406e+00, 1.62854249e+00,
       1.52950929e+00, 1.42664381e+00, 1.32478768e+00, 1.27554390e+00,
       1.21253926e+00, 1.15699224e+00, 1.14223561e+00, 1.08140859e+00,
       1.05831636e+00, 1.04167785e+00, 1.00857664e+00, 9.38895349e-01,
       9.11099176e-01, 8.55166277e-01, 8.19187164e-01, 7.94319149e-01,
       7.62904166e-01, 7.55881376e-01, 7.15453836e-01, 6.93704506e-01,
       6.81613005e-01, 6.28553779e-01, 6.20353194e-01, 5.66408946e-01,
       5.48157595e-01, 5.05362928e-01, 4.81325902e-01, 4.50461328e-01,
       4.06325635e-01, 3.74823658e-01, 3.45686134e-01, 2.72181785e-01,
       2.30627815e-01, 1.44735048e-01, 7.50311976e-02, 4.57077066e-02,
       2.07157442e-02, 1.18287831e-02, 6.43691242e-03, 3.91805002e-03,
       2.17377496e-03, 8.96794862e-05, 2.45714031e-06, 2.43711394e-06,
       2.39359459e-06, 2.37693907e-06, 2.35836338e-06, 2.34921429e-06,
      

In [20]:
s.shape

(500,)

In [21]:
# Unitary arrays (matrix)
# vh 
# Actually here have same result as the "u" 

In [22]:
vh = pd.DataFrame(vh)

In [23]:
vh

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.006387,-0.010837,-0.009057,-0.010602,-0.017506,-0.020772,0.002205,-0.007663,-0.006408,-0.021189,...,0.010054,0.018009,0.018255,0.030588,-0.014835,-0.015570,-0.003109,0.011545,-0.002565,-0.011286
1,0.023537,0.005218,0.006176,0.003314,0.042395,0.036854,0.018510,0.002920,0.009807,-0.021750,...,-0.011303,-0.011063,-0.017403,-0.003624,-0.026346,-0.035488,-0.035706,-0.017246,-0.008398,-0.008489
2,-0.004673,0.014919,0.027448,0.021919,0.023584,0.030860,-0.002765,0.024576,0.020967,0.101282,...,0.079818,0.086883,0.023343,-0.012814,0.033091,0.021553,0.000365,-0.037298,-0.042501,-0.023948
3,0.027282,-0.048530,0.044896,0.007954,0.022463,0.026459,-0.035689,-0.030301,0.013820,0.045942,...,0.024963,0.014741,-0.020335,-0.019253,0.031483,-0.036509,-0.074290,-0.015249,0.032443,0.089253
4,-0.048683,-0.016666,-0.030452,-0.000284,-0.041785,-0.024583,0.020820,-0.047374,0.000348,-0.007839,...,-0.018395,-0.045159,0.013354,-0.037162,-0.032157,0.071049,-0.008611,-0.027140,0.023240,0.056195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.011797,-0.070397,-0.008014,-0.041899,-0.006870,-0.010960,-0.011996,0.043915,-0.055611,0.022623,...,-0.054625,-0.031026,0.003804,-0.090840,0.031022,-0.064265,0.085795,0.050347,0.040020,-0.022778
496,-0.027347,-0.067255,0.006137,-0.013406,0.038308,-0.004481,0.039573,0.007133,0.009797,-0.093589,...,0.019867,-0.044339,-0.041987,-0.069452,-0.059265,-0.015115,-0.018858,-0.031000,0.013491,-0.001411
497,-0.005959,0.004390,0.099489,0.034253,-0.023583,0.028885,0.028907,-0.023617,0.042867,-0.032751,...,0.055534,0.009855,-0.038935,0.092464,0.001123,0.079277,0.029559,0.001908,0.037706,-0.057886
498,-0.039187,0.052487,0.025765,0.014198,0.044797,0.025664,-0.032950,0.007805,-0.028169,0.083690,...,-0.028993,0.027184,0.035850,0.011074,0.056673,0.016957,0.017190,-0.065878,-0.037748,-0.003453


In [24]:
vh.shape

(500, 500)

In [25]:
# LAMBDA
lambda_ = s

In [26]:
cumCntrb = np.cumsum(lambda_/len(lambda_))

In [27]:
lambd = lambda_

In [34]:
lambd = pd.DataFrame(lambd)

In [35]:
lambd

,0
0,7.415728e+00
1,4.877160e+00
2,2.917023e+00
3,2.263043e+00
4,2.091741e+00
...,...
495,9.459658e-09
496,8.485412e-09
497,3.743213e-09
498,3.519035e-09


In [29]:
# COEFFICIENT
a = np.sqrt(len(df_scl)*lambd)

In [30]:
# a
a.shape

(500,)

In [31]:
coefficient = np.divide(u, a.T)

In [32]:
coefficient.shape

(500, 500)

In [33]:
coefficient

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.000105,0.000477,-0.000122,0.000811,-0.001505,0.000063,-0.001167,0.000629,-0.000325,0.001204,...,-12.376775,10.382078,-12.884696,1.690135,-15.087133,5.424561,-13.276635,4.355762,-29.542362,-41202.373439
1,-0.000178,0.000106,0.000391,-0.001443,-0.000515,0.000089,0.000312,-0.000700,0.001253,-0.001811,...,-12.772763,-0.766002,-4.632076,-26.671157,17.284659,32.369348,-32.651312,-3.208731,39.569108,-41202.330175
2,-0.000149,0.000125,0.000719,0.001335,-0.000942,-0.002309,0.000356,-0.000370,0.001192,-0.001477,...,1.126346,-11.511614,-7.075241,-7.883159,16.271328,3.685136,2.979576,-72.722020,19.423568,-41202.344088
3,-0.000174,0.000067,0.000574,0.000236,-0.000009,-0.000804,0.000018,0.000073,0.002019,-0.002369,...,0.266576,5.497336,-4.848839,16.423661,-33.780983,19.265623,-6.508220,-25.037732,10.703982,-41202.350829
4,-0.000287,0.000859,0.000618,0.000668,-0.001292,0.001648,-0.001266,0.001282,-0.001864,-0.000639,...,1.164543,-25.253771,-11.146677,12.290213,-39.865854,3.158824,18.598059,17.238070,33.771808,-41202.331113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.000256,-0.000719,0.000564,-0.001085,0.002197,0.001235,0.002049,0.000149,0.001369,-0.003077,...,-9.216399,1.101597,-5.846034,-6.865274,11.673180,29.549571,-7.338150,-57.948372,12.783220,-41202.347740
496,-0.000051,-0.000723,0.000010,-0.002209,-0.000266,-0.001866,-0.001185,-0.001148,0.000779,-0.000153,...,-9.129216,-5.903220,16.769751,-2.298028,-28.239895,-39.449170,-9.155486,-21.606605,12.958875,-41202.351652
497,0.000190,-0.000349,-0.000977,-0.000453,-0.000839,-0.001562,-0.003574,0.000513,-0.001148,0.000184,...,6.469670,-3.966850,-8.414350,17.519548,25.051070,-23.149800,-15.050310,-1.394425,-49.664138,-41202.382560
498,-0.000042,-0.000170,-0.001113,0.000964,0.000719,-0.000604,-0.000384,-0.000127,-0.002323,0.000124,...,-16.393860,6.711271,13.039767,-7.341459,-18.191699,-18.401514,6.549914,-27.561371,-28.457365,-41202.371496


## Scipy SVD

In [36]:
from scipy.linalg import svd

In [37]:
U, s, VT = svd(K_c/len(df_scl))

In [41]:
U = pd.DataFrame(U)

In [42]:
U

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.006387,0.023537,-0.004673,0.027282,-0.048683,0.001987,-0.036125,0.017959,-0.008994,0.032167,...,-0.050937,0.041807,-0.043418,0.005216,-0.037156,0.011797,-0.027347,0.005959,-0.039187,-0.044721
1,-0.010837,0.005218,0.014919,-0.048530,-0.016666,0.002775,0.009661,-0.019979,0.034658,-0.048371,...,-0.052567,-0.003085,-0.015609,-0.082312,0.042567,0.070397,-0.067255,-0.004390,0.052487,-0.044721
2,-0.009057,0.006176,0.027448,0.044896,-0.030452,-0.072360,0.011022,-0.010565,0.032952,-0.039450,...,0.004636,-0.046356,-0.023842,-0.024329,0.040072,0.008014,0.006137,-0.099489,0.025765,-0.044721
3,-0.010602,0.003314,0.021919,0.007954,-0.000284,-0.025181,0.000550,0.002075,0.055837,-0.063263,...,0.001097,0.022137,-0.016339,0.050687,-0.083193,0.041899,-0.013406,-0.034253,0.014198,-0.044721
4,-0.017506,0.042395,0.023584,0.022463,-0.041785,0.051631,-0.039164,0.036588,-0.051554,-0.017074,...,0.004793,-0.101694,-0.037562,0.037930,-0.098179,0.006870,0.038308,0.023583,0.044797,-0.044721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.015570,-0.035488,0.021553,-0.036509,0.071049,0.038697,0.063405,0.004241,0.037852,-0.082169,...,-0.037931,0.004436,-0.019700,-0.021188,0.028748,0.064265,-0.015115,-0.079277,0.016957,-0.044721
496,-0.003109,-0.035706,0.000365,-0.074290,-0.008611,-0.058478,-0.036679,-0.032770,0.021545,-0.004079,...,-0.037572,-0.023772,0.056510,-0.007092,-0.069547,-0.085795,-0.018858,-0.029559,0.017190,-0.044721
497,0.011545,-0.017246,-0.037298,-0.015249,-0.027140,-0.048940,-0.110584,0.014648,-0.031744,0.004925,...,0.026626,-0.015974,-0.028354,0.054069,0.061694,-0.050347,-0.031000,-0.001908,-0.065878,-0.044721
498,-0.002565,-0.008398,-0.042501,0.032443,0.023240,-0.018927,-0.011876,-0.003614,-0.064240,0.003310,...,-0.067470,0.027025,0.043941,-0.022657,-0.044801,-0.040020,0.013491,-0.037706,-0.037748,-0.044721


In [39]:
s

array([7.41572790e+00, 4.87716031e+00, 2.91702285e+00, 2.26304325e+00,
       2.09174059e+00, 1.96410904e+00, 1.91524406e+00, 1.62854249e+00,
       1.52950929e+00, 1.42664381e+00, 1.32478768e+00, 1.27554390e+00,
       1.21253926e+00, 1.15699224e+00, 1.14223561e+00, 1.08140859e+00,
       1.05831636e+00, 1.04167785e+00, 1.00857664e+00, 9.38895349e-01,
       9.11099176e-01, 8.55166277e-01, 8.19187164e-01, 7.94319149e-01,
       7.62904166e-01, 7.55881376e-01, 7.15453836e-01, 6.93704506e-01,
       6.81613005e-01, 6.28553779e-01, 6.20353194e-01, 5.66408946e-01,
       5.48157595e-01, 5.05362928e-01, 4.81325902e-01, 4.50461328e-01,
       4.06325635e-01, 3.74823658e-01, 3.45686134e-01, 2.72181785e-01,
       2.30627815e-01, 1.44735048e-01, 7.50311976e-02, 4.57077066e-02,
       2.07157442e-02, 1.18287831e-02, 6.43691242e-03, 3.91805002e-03,
       2.17377496e-03, 8.96794862e-05, 2.45714031e-06, 2.43711394e-06,
       2.39359459e-06, 2.37693907e-06, 2.35836338e-06, 2.34921429e-06,
      

In [43]:
VT = pd.DataFrame(VT)

In [44]:
VT

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,-0.006387,-0.010837,-0.009057,-0.010602,-0.017506,-0.020772,0.002205,-0.007663,-0.006408,-0.021189,...,0.010054,0.018009,0.018255,0.030588,-0.014835,-0.015570,-0.003109,0.011545,-0.002565,-0.011286
1,0.023537,0.005218,0.006176,0.003314,0.042395,0.036854,0.018510,0.002920,0.009807,-0.021750,...,-0.011303,-0.011063,-0.017403,-0.003624,-0.026346,-0.035488,-0.035706,-0.017246,-0.008398,-0.008489
2,-0.004673,0.014919,0.027448,0.021919,0.023584,0.030860,-0.002765,0.024576,0.020967,0.101282,...,0.079818,0.086883,0.023343,-0.012814,0.033091,0.021553,0.000365,-0.037298,-0.042501,-0.023948
3,0.027282,-0.048530,0.044896,0.007954,0.022463,0.026459,-0.035689,-0.030301,0.013820,0.045942,...,0.024963,0.014741,-0.020335,-0.019253,0.031483,-0.036509,-0.074290,-0.015249,0.032443,0.089253
4,-0.048683,-0.016666,-0.030452,-0.000284,-0.041785,-0.024583,0.020820,-0.047374,0.000348,-0.007839,...,-0.018395,-0.045159,0.013354,-0.037162,-0.032157,0.071049,-0.008611,-0.027140,0.023240,0.056195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.011797,-0.070397,-0.008014,-0.041899,-0.006870,-0.010960,-0.011996,0.043915,-0.055611,0.022623,...,-0.054625,-0.031026,0.003804,-0.090840,0.031022,-0.064265,0.085795,0.050347,0.040020,-0.022778
496,-0.027347,-0.067255,0.006137,-0.013406,0.038308,-0.004481,0.039573,0.007133,0.009797,-0.093589,...,0.019867,-0.044339,-0.041987,-0.069452,-0.059265,-0.015115,-0.018858,-0.031000,0.013491,-0.001411
497,-0.005959,0.004390,0.099489,0.034253,-0.023583,0.028885,0.028907,-0.023617,0.042867,-0.032751,...,0.055534,0.009855,-0.038935,0.092464,0.001123,0.079277,0.029559,0.001908,0.037706,-0.057886
498,-0.039187,0.052487,0.025765,0.014198,0.044797,0.025664,-0.032950,0.007805,-0.028169,0.083690,...,-0.028993,0.027184,0.035850,0.011074,0.056673,0.016957,0.017190,-0.065878,-0.037748,-0.003453


reference 

- scipy, [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.svd.html)
- numpy, [here](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)
- matlab, [here](https://www.mathworks.com/help/matlab/ref/double.svd.html#bu2_0hq-U)
- other, [here](https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/)